## [T246439](https://phabricator.wikimedia.org/T246439) - High proportion of edit conflicts seem to come from new article creation
Nearly a third of edit conflicts are recorded as happening during new article creation.  We need to investigate this, determine what exactly the user experiences, if anything.

In [2]:
conflicts = spark.read.parquet("/user/awight/edit-conflicts/conflict_rev_details").cache()
conflicts.count()

83309

In [3]:
from pyspark.sql.functions import col
new_article = conflicts.filter(col("base_rev_id") == 0)

In [9]:
new_article\
    .select("conflict_timestamp", "wiki", "page_namespace", "page_title")\
    .orderBy("conflict_timestamp")\
    .show(15, truncate=False)

+-------------------+-----------+--------------+----------------------------------------------------------------------+
|conflict_timestamp |wiki       |page_namespace|page_title                                                            |
+-------------------+-----------+--------------+----------------------------------------------------------------------+
|2020-04-01 00:50:01|ptwiki     |14            |!Fantoches de Madragoama                                              |
|2020-04-01 01:19:40|jawiki     |0             |ギャバード                                                            |
|2020-04-01 01:20:20|commonswiki|14            |Portrait of a Woman by Jean-Baptiste Greuze – Wallace Collection      |
|2020-04-01 01:59:12|arwiki     |14            |مؤرخون هنود في القرن 15                                               |
|2020-04-01 04:36:05|zhwiki     |2             |AndyAndyAndyAlbert/沙盒/Q                                             |
|2020-04-01 04:46:27|ptwiki     |14            

These aren't popular pages, many were only ever edited once at timestamps exactly matching the conflicts above.  They happen in clusters around similar article titles, so it seems likely this is a small number of users who self-conflict for a technical reason like a rogue gadget or userscript.